# Confocal Microscopy Multi Label Image Classification - Transfer Learning & Regularization

In [ ]:
import os
import time
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset                  # TensorDataset !!! we import Dataset for custom 'ImageFolder'
import torchvision
# from torchvision.datasets import ImageFolder # MNIST, CIFAR10 etc
# from torchvision.datasets.utils import download_url
from torchvision.utils import make_grid                                         # save_image 
import torchvision.transforms as T # ToTensor, Compose, Normalize, RandomCrop, RandomResizedCrop, RandomHorizontalFlip, RandomRotate, RandomErasing, ColorJitter
import torchvision.models as models                                             # pretreinde models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')

device = device()

def todevice(data_model, device):
  if isinstance(data_model, (list, tuple)):
    return [todevice(i, device) for i in data_model]
  return data_model.to(device, non_blocking=True)

class DataDeviceLoader():
  def __init__(self, dataloader, device):
    self.dataloader = dataloader   
    self.device = device

  def __iter__(self):
    for batch in self.dataloader:
      yield todevice(batch, self.device)
  
  def __len__(self):
    return len(self.dataloader)

In [ ]:
# ! pip install -q kaggle                                       
# ! mkdir ~/.kaggle                                                
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download -c jovian-pytorch-z2g                         # https://www.kaggle.com/c/jovian-pytorch-z2g     
# # ! kaggle datasets download ikarus777/best-artworks-of-all-time
# # ! unzip jovian-pytorch-z2g

In [ ]:
datadir = './data'
traindir = datadir + '/train'
testdir = datadir + '/test'

train_valid_info = pd.read_csv(traindir)
testinfo = pd.read_csv(testdir)

### 1. Custom random_split (into training and validation info) by creating masks

np.random.seed(42)
mask = np.random.rand(len(train_valid_info)) < 0.9              # it gives uniform distribution including zeros, so there will be about 1/10 zeros
traininfo_dataframe = train_valid_info[mask].reset_index()      # all zeros are not taken 1/10 and then indexing is reset for new dataframe       
validinfo_dataframe = train_valid_info[~mask].reset_index()     # mask reversed (only those are taken that previously were zeros)


### 2. Custom Lable Handling (string lable to vector and vice versa)

labels = {
    0: 'Mitochondria',
    1: 'Nuclear bodies',
    2: 'Nucleoli',
    3: 'Golgi apparatus',
    4: 'Nucleoplasm',
    5: 'Nucleoli fibrillar center',
    6: 'Cytosol',
    7: 'Plasma membrane',
    8: 'Centrosome',
    9: 'Nuclear speckles'
}

def encode_tovector(stringlabel):               # like '1' , '2', '5 8' etc.
  vector = torch.zeros(10)                                                      # if we have 10 classes 0-9 are position labels
  for labelpart in str(stringlabel).split(' '):                                 # if string consists of many label parts
    vector[int(labelpart)] = 1
  return vector

def decode_tostringlabel(vectorlabel, namelabels=False, treshold=0.5):          # namelabels = labels   (defined above)
  stringlabels =[]
  for i, encoding in enumerate(vectorlabel):
    if (encoding >= treshold):
      if namelabels:
        stringlabels.append(namelabels[i] + ' ' + str(i))
      else:
        stringlabels.append(str(i))
  return ' '.join(stringlabels)                                                 # because syntax is ' '.join([...])


### 3. Custom Dataclass like ImageFolder (take lables from csv and add those lables as vectors to pictures + put them in transform)

class ImageLabelerTransformizer(Dataset):                                       # Dataset - requires this structure of the class!
  def __init__(self, infodataframe, datapath=datadir, transformizer=False):
    self.infodataframe = infodataframe
    self.datapath = datapath
    self.transformizer = transformizer
  
  def __len__(self):
    return len(self.infodataframe)

  def __getitem__(self, rowindex):
    onepictureinfo = self.infodataframe.loc[rowindex]                           # genarally with rows with numerical names we use iloc() but here we have string versions like '0', '1' etc
    imagename, stringlabel = onepictureinfo['Image'], onepictureinfo['Label']
    imagefullpathname = self.datapath + '/' + str(imagename) + '.png'
    image = Image.open(imagefullpathname)
    if self.transformizer:
      image = transformizer(image)
    return image, encode_tovector(stringlabel)


### Standart part

RGB_mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                              # you should check statistics are given with data
train_tensorized = T.Compose([T.RandomCrop(512, padding=8, padding_mode='reflect'),
                              #  T.RandomResizeCrop(256, scale=(0.5, 0.9), ratio=(1, 1)),  # T.CenterCrop(32), Ts.Resize(32)
                               T.RandomHorizontalFlip(),
                               T.RandomRotation(10),                                       # 10 degrees
                              #  T.CollorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                               T.RandomErasing(), 
                               T.ToTensor(),
                              #  T.Normalize(*RGB_mean_std, inplace=True)
                               ])      # to make operation in place or რომ არსებულს ზევიდან გადააწეროს, არ ვიცი
validortest_tensorized = T.Compose([T.ToTensor(), 
                              # T.Normalize(*RGB_mean_std)
                              ]) 

traindataset = ImageLabelerTransformizer(infodataframe=traininfo_dataframe, datapath=datadir, transformizer=train_tensorized)         
validset = ImageLabelerTransformizer(infodataframe=validinfo_dataframe, datapath=datadir, transformizer=validortest_tensorized)  
testset =  ImageLabelerTransformizer(infodataframe=testinfo, datapath=datadir, transformizer=validortest_tensorized)              

batchsize = 128
trainloader = DataLoader(traindataset, batch_size=batchsize, shuffle=True, num_workers=2, pin_memory=True)  
validloader = DataLoader(validset, batch_size=batchsize*2, num_workers=2, pin_memory=True)
testloader = DataLoader(testset, batch_size=batchsize*2, num_workers=2, pin_memory=True)

trainload = DataDeviceLoader(trainloader, device)
validload = DataDeviceLoader(validloader, device) 
trainload = DataDeviceLoader(testloader, device) 

In [ ]:
# image, _ = somedataset[0]
# print(f"image shape is {image.shape}") 

def gridofimages(trainload, invert=False, number=64):          # without for+break images = next(iter(trainload))
  for images, _ in trainload:
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    data = (1-images[: number]) if invert else images[: number]
    plt.imshow(make_grid(data.cpu().detach(), normalize=True, nrow=8).permute(1, 2, 0))   #images.cpu() because we used dataload (which is on gpu, if gpu is available) not dataloader
    break
                                                  # normalize=True reverses/unnormalizes what transforms.Normalize has done
gridofimages(trainload, invert=True)

In [ ]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold                    #(out of 10) All other entries below treshold became None
    label = label > threshold                    #(out of 10) All other entries are zero already but we still want them to become None

    TruePositive = (prob & label).sum(axis=1).float()
    TrueNegative = ((~prob) & (~label)).sum(axis=11).float()                    # Just for knowledge - because we do not use this
    FalsePositive = (prob & (~label)).sum(axis=1).float()
    FalseNegative = ((~prob) & label).sum(axis=1).float()

    precision = torch.mean(TruePositive/ (TruePositive+ FalsePositive + 1e-12))
    recall = torch.mean(TruePositive/ (TruePositive+ FalseNegative + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)   #harmonic mean 2/(1/precision + 1/recall)
    return F2.mean(axis=0)                                         


class LossPart(nn.Module):
  def trainloss(self, batch):
    images, lables = batch
    out = self(images)
    loss = F.binary_cross_entropy(out, lables)
    return loss

  def validloss(self, batch):
    images, lables = batch
    out = self(images)
    loss = F.binary_cross_entropy(out, lables)
    score = F_score(out, lables)
    return {'score': score, 'loss': loss.detach()}

  def epochend(self, epochoutputs):
    epochlosses = [batch['loss'] for batch in epochoutputs]
    epochaverageloss = torch.stack(epochlosses).mean()
    epochscores = [batch['score'] for batch in epochoutputs]
    epochaveragescore = torch.stack(epochscores).mean()
    return {'epochloss' : epochaverageloss.item(), 'epochscore' : epochaveragescore.item()}


class ForwardPart(LossPart):                                                    
  def __init__(self):
    super().__init__()                                                          
    self.model = models.resnet34(pretreined=True)
    self.lastinput = self.model.fc.in_features                                  # fc is last leyer and in_features input features
    self.lastoperation = nn.Linear(self.lastinput, 10)                          # as there are 10 classes to classify

  def forward(self, xbatch):
    return torch.sigmoid(self.model(xbatch))  # torch.sigmoid is basically same as torch.nn.Sigmoid, first is function, second is class callable like function

  def train_lastleyer(self):
    for parameter in self.model.parameters():
      parameter.require_grad = False
    for parameter in self.model.fc.parameters():    # in the pretrained model we took fc is last operation / 'layer' name
      parameter.require_grad = True

  def train_allleyers(self):
    for parameter in self.model.parameters():
      parameter.require_grad = True
    for parameter in self.model.fc.parameters():   
      parameter.require_grad = True


model = todevice(ForwardPart(), device)
model                                               # Just to see what structure has the model

In [ ]:
@torch.no_grad()
def evaluate(model, valid_or_testload):
  model.eval()
  epochoutputs = [model.validloss(batch) for batch in valid_or_testload]
  epochresult = model.epochend(epochoutputs)
  return epochresult

def fit(model, trainload, validload, max_lr, epochs, weight_decay=0, clip_grad=None, optim=torch.optim.Adam):
  torch.cuda.empty_cache()
  history = []
  optimizer = optim(model.parameters(), max_lr, weight_decay=weight_decay)
  scedule = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(trainload))
  
  for epoch in range(epochs):
    model.train()
    learning_rates = []
    training_losses = []

    for batch in trainload:
      loss = model.trainloss(batch)
      training_losses.append(loss)
      loss.backward()
      if clip_grad:
        nn.utils.clip_grad_value_(model.parameters(), clip_grad)                # clips weights from previous time
      optimizer.step()                                                          # here will be used weight_decay=weight_decay
      optimizer.zero_grad()
      learning_rates.append([i['lr'] for i in optimizer.param_groups])
      scedule.step()

    epochresult = evaluate(model, validload)
    epochresult['epoch_trainloss'] = torch.stack(training_losses).mean().item() # creates new key-value pair in dictionary
    epochresult['lr'] = learning_rates                                          
    history.append(epochresult)
  
  return history

In [ ]:
max_lr = 0.01
epochs = 8
weight_decay = 1e-4
clip_grad = 0.1
optim = torch.optim.Adam

model.train_lastleyer()             #first we train only last / new layer

training = []

%time
training += fit(model, trainload, validload, max_lr, epochs, weight_decay, clip_grad, optim)

In [ ]:
model.train_allleyers()             #lastly several times, less epochs or not, we train all layers

training += fit(model, trainload, validload, max_lr, epochs, weight_decay, clip_grad, optim)

In [ ]:
scores = [s['epochscore'] for s in training]
validlosses = [loss['epochloss'] for loss in training]
trainlosses = [loss.get('epoch_trainloss') for loss in training]
learningrates = np.concatenate([lr.get('lr', []) for lr in training])
plt.plot(scores, '-mo')
plt.plot(validlosses, '-bo')
plt.plot(trainlosses, '-co')
# plt.plot(learningrates, '-yo')                        #plot separately as scale is completely different and nothing will be seen in same graph, xlabel=batch
plt.legend(['score', 'validloss', 'trainloss', 'lrs'])
plt.xlabel('epoch')
plt.ylabel('value')
plt.title('learning Performance');

In [ ]:
def predict(model, image):                                           # here we put images that are not normalized, so row image from testset not fromtestload!!
  image, label = image
  uimage = todevice(image.unsqueeze(0), device)
  predicted_vectorlabel = model(uimage)
  predicted_stringlabel = decode_tostringlabel(predicted_vectorlabel, namelabels=True)
  print(f'predicted label is {predicted_stringlabel}, actual label is {label}')
  plt.imshow(image.permute(1, 2, 0)) 
  plt.axis('off');

predict(testset[7])

In [ ]:
torch.save(model.state_dict(), 'microscopy.pth')
new_model = ForwardPart()
new_model.load_state_dict(torch.load('microscopy.pth')) 